In [ ]:
import pandas as pd
import pickle

def find_coordinates():
    data = pd.read_csv('../surveys/Three Forks CD/2010 DMR Three Forks Assessment CD/LogsUsedAllWells.DAT', 
            sep="\s+", 
            skiprows=1, 
            usecols=[i for i in range(8)], 
            names = ['API','DEPTH','POROSITY','RT','RW','SW','So', 'TEMP(F)'])
    
    # get all the unique api no's, then go to drillinginfo and extract the coordinates
    api_nos = data['API'].unique()
    
    # read in and create a dictionary of surface coordinates per well for later reference
    envernus = pd.read_csv('../datasets/three forks wells.CSV')
    envernus = envernus.rename(columns={
        'Surface Hole Latitude (WGS84)': 'surface_latitude',
        'Surface Hole Longitude (WGS84)': 'surface_longitude',
        'True Vertical Depth': 'TVD'
    })  

    coordinate_dict = {
        row['API14']: {
            'latitude':row['surface_latitude'],
            'longitude':row['surface_longitude'],
            'tvd':row['TVD']
        }
        for _, row in envernus.iterrows()
    }

    # once created, write to housekeeping directory for later use.    
    with open('../housekeeping/three_forks_coordinates.pkl', 'wb') as handle:
        pickle.dump(coordinate_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

find_coordinates()

In [ ]:
import glob
import pickle

def generate_LAS_file_list(directory):
    return glob.glob(directory)

def pickle_LAS(file):
    with open(file, 'r') as f:
        lines = list(filter(None,[line.strip().split() for line in f.read().splitlines() if line]))
    
    # find what we care about: the apino, and the data delimiter, ~A.
    for idx, line in enumerate(lines):

        if 'API' in line[0]:
            api_no = line[1]
            
        # this will always be found after we know the api_no, so break.
        if '~A' in line[0]:
            data_start = idx+1
            break

    data = pd.DataFrame(lines[data_start:],
                        columns = ['DEPTH', 'DENSITY_POROSITY', 'RT','RW', 'SW','TEMP(F)'])
    
    # once created, write to housekeeping directory for later use.    
    with open(f'../datasets/three forks/{api_no}.pkl', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

LAS_files = generate_LAS_file_list('../surveys/Three Forks CD/2010 DMR Three Forks Assessment CD/LAS Files/*')

for file in LAS_files:
    pickle_LAS(file)

In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def plot_3fork_logs(file):
    with open(file, 'rb') as handle:
        data = pickle.load(handle).astype(float)
    
    # i know there will always be the same count of curves just because 
    # thats the way the data's structured, but just want some flexibility later.
    data = data.set_index(data.columns[0])
    curve_count = len(data.columns)
    
    # find the min, max of first column to use gradient-area fill plotting.
    def area_fill(data, col, fig, count):

        ref = data[col]
        _min, _max = ref.min(), ref.max() 
        span = abs(_max - _min)
        cmap = plt.get_cmap('viridis')
        color_index = np.arange(_min, _max, span/100)

        axs[count].plot(ref, data.index, color='black')

        # given the density-porosity, visualize the 
        for index in sorted(color_index):
            index_value = (index - _min) / span
            color = cmap(index_value)
        
            axs[count].fill_betweenx(data.index, _max, ref, 
                            where=ref>=index,
                            color=color)

            axs[count].set(xlabel = col)
            axs[count].xaxis.set_label_position('top')
            axs[count].tick_params(axis='x', rotation=315)

    fig, axs = plt.subplots(1,3,figsize=(24,24),
                            sharey=True)

    # only need to invert one y-axis since they're shared.
    axs[0].invert_yaxis()
    
    count = 0
    for col in ['DENSITY_POROSITY', 'RT','SW']:
         area_fill(data, col, axs, count)
         count+=1        
    
plot_3fork_logs('../datasets/three forks/33007009970000.pkl')


In [ ]:
import pickle
import pandas as pd
import glob
import matplotlib.pyplot as plt
from matplotlib import interactive

interactive = True
%matplotlib qt

def verify_3fork_data():
    
    # first, read in our dictionary with longitudes and latitudes
    with open('../housekeeping/three_forks_coordinates.pkl', 'rb') as handle:
        coordinates = pickle.load(handle)
    
    # create sets out of the two lists, ensure the types are same, find intersection
    # between sets to get the api14's we need.
    _t = set(coordinates.keys())
    directories = glob.glob('../datasets/three forks/*')
    _d = set([int(d.split('\\')[1].split('.')[0]) for d in directories])
    
    # valid.
    apis = list(_t.intersection(_d))

    coordinates = {k:v for k,v in coordinates.items() if k in apis}
    
    return_coordinates = coordinates.copy()
    for key in coordinates:
        with open(f'../datasets/three forks/{key}.pkl', 'rb') as handle:
            data = pickle.load(handle)
            return_coordinates[key]['max_depth'] = data['DEPTH'].iloc[-1]
            return_coordinates[key]['min_depth'] = data['DEPTH'].iloc[0]
            
    # once created, write to housekeeping directory for later use.    
    with open('../housekeeping/three_forks_coordinates_depth.pkl', 'wb') as handle:
        pickle.dump(return_coordinates, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    return apis

api14_nos = verify_3fork_data()

def generate_3forks_surface(api_nos):
    
    # first, read in our dictionary with longitudes and latitudes
    with open('../housekeeping/three_forks_coordinates_depth.pkl', 'rb') as handle:
        _coordinates = pickle.load(handle)

    dataframe = pd.DataFrame.from_dict(_coordinates, orient='index').astype(float)
    
    X, Y, Z_top, Z_bot = dataframe['latitude'], dataframe['longitude'], dataframe['min_depth'], dataframe['max_depth']

    # to Add a color bar which maps values to colors.
    fig = plt.figure(figsize=(20,12))
    ax = fig.gca(projection='3d')
    surf=ax.plot_trisurf(Y, X, Z_top, cmap=plt.cm.viridis,
                        edgecolor='none', linewidth=0, antialiased=False)
    cbar = fig.colorbar( surf, shrink=0.5, aspect=5, )
    cbar.set_label('Three Forks Top (ft)')
    ax.set_xlabel('Bottom Hole Latitude (WGS84)')
    ax.set_ylabel('Bottom Hole Longitude (WGS84)')
    ax.set_zlabel('Three Forks Top (ft)')
    ax.invert_zaxis()

    # to Add a color bar which maps values to colors.
    fig = plt.figure(figsize=(20,12))
    ax = fig.gca(projection='3d')
    surf=ax.plot_trisurf(Y, X, Z_bot, cmap=plt.cm.viridis,
                        edgecolor='none', linewidth=0, antialiased=False)
    cbar = fig.colorbar( surf, shrink=0.5, aspect=5, )
    cbar.set_label('Three Forks Top (ft)')
    ax.set_xlabel('Bottom Hole Latitude (WGS84)')
    ax.set_ylabel('Bottom Hole Longitude (WGS84)')
    ax.set_zlabel('Three Forks Top (ft)')
    ax.invert_zaxis()

    plt.show()    

generate_3forks_surface(api14_nos)

In [158]:
# create folders for all these damn trajectories. im downloading them by hand bro..
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

with open('../housekeeping/3fork trajectories.txt') as f:
    lines = f.read().splitlines()
    print(lines)
# for api in lines:
#     createFolder(f'../datasets/three forks/trajectories/{api}')

['33007011850000', '33007014220000', '33007014300000', '33007014390000', '33007014480000', '33007014590000', '33007014690000', '33007015020000', '33007015070000', '33007015130000', '33007015420000', '33007015540000', '33007015630000', '33007015750000', '33007015800000', '33013013830000', '33013014020000', '33013014140000', '33013014310000', '33013014340000', '33023004510000', '33023004550000', '33023004590000', '33023004620000', '33023004630000', '33023004660000', '33023004800000', '33023004890000', '33023005040000', '33025005890000', '33025005970000', '33025006690000', '33025007160000', '33025007280000', '33033002390000', '33053023570000', '33053025320000', '33053025390000', '33053025500000', '33053025520000', '33053025890000', '33053026470000', '33053026690000', '33053026720000', '33053026840000', '33053027040000', '33053027200000', '33053027410000', '33053027480000', '33053027570000', '33053027600000', '33053027690000', '33053027940000', '33053028060000', '33053028080000', '33053028